# Imports básicos para todas as análises

In [1]:
# Desativar as mensagens de 'warning' que ficam poluindo o output aqui no Colab

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Importações dos módulos necessários para autenticar na minha conta Google e acessar o Drive
import  pandas as pd,               \
        numpy as np,                \
        matplotlib.pyplot as plt,   \
        requests as rt,             \
        seaborn as sns,             \
        xml.etree.ElementTree as ET

from io               import BytesIO
from tqdm             import tqdm
from matplotlib.pylab import rcParams
from pandas.plotting  import register_matplotlib_converters

from sktime.utils.plotting import plot_series
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

# Ajustes feitos para geração e criação de gráfico
rcParams['figure.figsize'] = 15, 6

# Tratar conversões de DateTime entre o Pandas e o Matplotlib
register_matplotlib_converters()

### Funções úteis

In [3]:
# import calendar
# import datetime
# import json
# import os
# import requests
# import xml.etree.ElementTree as ET
# from tqdm import tqdm

# def get_data_telemetrica(codEstacao, dataInicio, dataFim): #, save=False

#     # 1. Fazer a requisião ao servidor e pegar a árvore e a raiz dos dados
#     params = {'codEstacao':codEstacao, 'dataInicio':dataInicio, 'dataFim':dataFim}
#     server = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/DadosHidrometeorologicos'
#     response = requests.get(server, params)
#     tree = ET.ElementTree(ET.fromstring(response.content))
#     root = tree.getroot()

#     # 2. Iteração dentro dos elementos do XML procurando os dados que são disponibilizados para a estação
#     list_vazao = []
#     list_data = []
#     list_nivel = []
#     list_chuva = []

#     for i in tqdm(root.iter('DadosHidrometereologicos')):

#         data = i.find('DataHora').text
#         try:
#             vazao = float(i.find('Vazao').text)
#         except TypeError:
#             vazao = i.find('Vazao').text

#         try:
#             nivel = float(i.find('Nivel').text)
#         except TypeError:
#             nivel = i.find('Nivel').text

#         try:
#             chuva = float(i.find('Chuva').text)
#         except TypeError:
#             chuva = i.find('Chuva').text

#         list_vazao.append(vazao)
#         list_data.append(data)
#         list_nivel.append(nivel)
#         list_chuva.append(chuva)

#     df = pd.DataFrame([list_data, list_nivel, list_chuva, list_vazao]).transpose()
#     df.columns = ['Data', 'Nivel', 'Chuva', 'Vazao']

#     df = df.sort_values(by='Data')
#     df = df.set_index('Data')
#     df.index = pd.to_datetime(df.index)

#     # if save == True:
#     #     df.to_excel(codEstacao+'_dados.xlsx')

#     return df

In [4]:
def sliding_window(data, n_lags=1, n_targets=1, dropnan=True):
	"""
	Rearranja a série temporal como um dataset para aprendizado supervisionado.
	Argumentos:
		data: Sequência de observações. Pode ser list ou NumPy array
		n_lags: Número de lags das observações que serão usadas como input (X).
		n_targets: Número de observações que serão usadas como output (y).
		dropnan: Booleano pra dropar as linhas que ficarem com NaN .
	Retorna:
		Um DataFrame como uma série ajustada para aprendizado supervisionado.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]

	df = pd.DataFrame(data)
	cols, names = list(), list()

	# As lags (t-n, ... t-1)
	for i in range(n_lags, 0, -1):
		cols.append(df.shift(i))
		names += [('X_%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

	# Prever uma sequência (t, t+1, ... t+n)
	for i in range(0, n_targets):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('y_%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('y_%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

	# Junta tudo
	agg = pd.concat(cols, axis=1)
	agg.columns = names

	# Dropa as linhas com NaN
	if dropnan:
		agg.dropna(inplace=True)

	return agg # IMPORTANTE!!!! O retorno sai com as colunas com nome zuado. Tem que renomear o DataFrame resultante depois.

In [5]:
# O corte realizado por esse método é simples: pega do início da série até train_size e coloca num dataframe para treino
# O que resta do corte anterior, é colocado no dataframe para teste
def split_train_test(df, train_size=0.7):

  # Tamanho total da série
  size = df.shape[0]

  # Tamanho do treino
  t_size = int(size * train_size)

  train_data = df.iloc[0:t_size]
  validation_data = df.iloc[t_size:]

  return train_data, validation_data

# Médio Jequitinhonha

### Criando o objeto 'dataset' para trabalho

In [64]:
estacao_principal = "./estacoes_medio/planilhas_ajustadas/principal_vazao_54195000.csv"
df_left = pd.read_csv(estacao_principal, sep='\t', index_col=0, header=0, parse_dates=['data'])

outras_estacoes = ["./estacoes_medio/planilhas_ajustadas/chuva_1642007.csv",
                   "./estacoes_medio/planilhas_ajustadas/chuva_1642008.csv",
                   "./estacoes_medio/planilhas_ajustadas/chuva_1642027.csv",
                   "./estacoes_medio/planilhas_ajustadas/cota_54150000.csv",
                   "./estacoes_medio/planilhas_ajustadas/cota_54193000.csv",
                   "./estacoes_medio/planilhas_ajustadas/cota_54195000.csv",
                   "./estacoes_medio/planilhas_ajustadas/telemetric_1642041.csv",
                   "./estacoes_medio/planilhas_ajustadas/telemetric_54140000.csv",
                   "./estacoes_medio/planilhas_ajustadas/telemetric_54150001.csv",
                   "./estacoes_medio/planilhas_ajustadas/vazao_54140000.csv",
                   "./estacoes_medio/planilhas_ajustadas/vazao_54150000.csv",
                   "./estacoes_medio/planilhas_ajustadas/vazao_54193000.csv"]

df_list = []
for f in outras_estacoes:
    df_list.append(pd.read_csv(f, sep='\t', index_col=0, header=0, parse_dates=['data']))

# df_merged = pd.merge(df_principal, df_secundaria, how="left", on="data", suffixes=[None, '_r'])

# df_merged[df_merged.index.year == 2015].to_clipboard(sep='\t')
# df_merged.dropna()
    
# df_left.to_clipboard(sep='\t')

# df_left.merge(df_right, how='left', on='data')
    
for df in df_list:
    df_result = df_left.merge(df, how='left', on='data', suffixes=(None, '_r'))
    df_left = df_result

df_result.to_clipboard(sep='\t')

In [ ]:
def criar_dataset(estacao_principal, outras_estacoes):
    df_principal = pd.read_csv(estacao_principal, sep='\t', index_col=0, header=0, parse_dates=['data'])
    
    df_outras = []
    for f in outras_estacoes:
        df_outras.append(pd.read_csv(f, sep='\t', index_col=0, header=0, parse_dates=['data']))


# Baixo Jequitinhonha